# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

# Import Dataset

In [ ]:
!pip install kaggle, kagglehub

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
import kagglehub, kaggle

In [ ]:
# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")
print("Path to dataset files:", path)

# VGG16 Model Transfer Learning

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top=False,
    input_shape=(150, 150, 3),
    name='conv_base',
)
conv_base.trainable = False
conv_base.summary()

# Transfer Learning Model

In [ ]:
model = Sequential()

## Add convolutional base
model.add(conv_base)

## Add Fully Connected Layers
model.add(Flatten(name='flatten_layer'))

## Hidden layers
model.add(Dense(units=256, activation='relu', name='fc_layer_1'))

## Output layers
model.add(Dense(units=1, activation='sigmoid', name='output_layer'))

In [ ]:
model.name = 'vgg16_transfer_learned'
model.summary()

# Prepare data for model training

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
# generators
train_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    sheer_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_data = train_data_generator.flow_from_directory(
    directoey='Cat_Dog_Dataset/train',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

validation_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
validation_data = validation_data_generator.flow_from_directory(
    directoey='Cat_Dog_Dataset/train',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
## Normalize dataset for training

def preprocess(image, label):
    image = tensorflow.cast(image/255.0, tensorflow.float32)
    return image, label

train_data = train_data.map(preprocess)
validation_data = validation_data.map(preprocess)

# Start Training

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(train_data, epochs=10, validation_data=validation_data).history

# Plot Learning Curve

In [ ]:
plt.plot(history['loss'], label='train loss', color='red')
plt.plot(history['val_loss'], label='val loss', color='green')
plt.legend()
plt.show()
plt.savefig('Feature_Extraction_With_Data_Aug/loss.png')

plt.plot(history['accuracy'], label='train accuracy', color='red')
plt.plot(history['val_accuracy'], label='val accuracy', color='green')
plt.legend()
plt.show()
plt.savefig('Feature_Extraction_With_Data_Aug/accuracy.png')

# Save Model

In [ ]:
model.save('Feature_Extraction_With_Data_Aug/cat_dog_cls_vgg16_transfer.keras')